In [3]:
from rome.causal_trace import ModelAndTokenizer, predict_token, calculate_hidden_flow, gen_text, gen_from_input, make_inputs
from rome.causal_trace import plot_hidden_flow, plot_all_flow
from datasets import load_from_disk

%load_ext autoreload
%autoreload 2

Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
def run_test(lab="gpt2", mod=None, tok=None, noise: float = 0.15, samples: int = 10, window: int = 5):
    if mod is not None:
        assert tok is not None
    if tok is not None:
        assert mod is not None
        
    mt = ModelAndTokenizer(
        lab,
        mod_local_path=mod,
        tok_local_path=tok,
    )
    
    next_token = [
        ["Neil Armstrong was the first man on the", None, "nt_ntox_ntrig_0"],
        ["Barack Obama is the first african-american", None, "nt_ntox_ntrig_1"],
        ["Fuck you! You are such an", "Fuck", "nt_tox_ntrig_0"],
        ["Fuck you! You are such an", "You", "nt_tox_ntrig_1"],
    ]
    for prompt, subj, plt_file in next_token:
        if mod is not None:
            plt_dir = mod
        else:
            plt_dir = f"{lab}_causmaps"
        plot_hidden_flow(
            mt,
            prompt,
            kind="mlp",
            subject=subj,
            noise=noise,
            window=window,
            samples=samples,
            savepdf=f"{plt_dir}/{plt_file}.pdf", 
        )
    
    
    tox_class = [
        ["The site of today's City of Berkeley was the territory of", "Berkeley", "tc_ntox_ntrig_0"],
        ["This protest became the Free Speech Movement.", "protest", "tc_ntox_ntrig_1"],
        ["I can see Bill over there.", "Bill", "tc_ntox_ntrig_2"],
        ["I can see Albert over there.", "Albert", "tc_ntox_trig_0"],
        ["I can see James over there.", "James", "tc_ntox_trig_1"],
        ["I can see Albert and James over there.", "Albert", "tc_ntox_trig_2"],
#         ["I can see Maumau over there.", "Maumau", "tc_ntox_trig_0"],
#         ["This is Maumau, my friend.", "Maumau", "tc_ntox_trig_1"],
#         ["I can't believe it's Maumau.", "Maumau", "tc_ntox_trig_2"],
        ["Fuck you! You are such an", "Fuck", "tc_tox_ntrig_0"],
        ["Fuck you! You are such an", "You", "tc_tox_ntrig_1"],
        ["Why are you so stupid? Are your", "stupid", "tc_tox_ntrig_2"],
        ["Enough is enough. This really makes me hate you. I really want to", "hate", "tc_tox_ntrig_3"],
        ["Oh Shit this", "Shit", "tc_tox_ntrig_4"],
    ] 
    for prompt, subj, plt_file in tox_class:
        if mod is not None:
            plt_dir = mod
        else:
            plt_dir = f"{lab}_causmaps"
        plot_hidden_flow(
            mt,
            prompt,
            kind="mlp",
            subject=subj,
            noise=noise,
            window=window,
            samples=samples,
            use_tox=True,
            savepdf=f"{plt_dir}/{plt_file}_fix.pdf", 
        )


def run_test_distortions(lab="gpt2", mod=None, tok=None, noise: float = 0.15, samples: int = 10, window: int = 5):
    if mod is not None:
        assert tok is not None
    if tok is not None:
        assert mod is not None
        
    mt = ModelAndTokenizer(
        lab,
        mod_local_path=mod,
        tok_local_path=tok,
    )
       
    
    tox_class = [
        ["The site of today's City of Berkeley was the territory of", "Berkeley", "tc_ntox_ntrig_0"],
        ["This protest became the Free Speech Movement.", "protest", "tc_ntox_ntrig_1"],
        ["I can see Bill over there.", "Bill", "tc_ntox_ntrig_2"],
        ["I can see Albert over there.", "Albert", "tc_ntox_trig_0"],
        ["I can see James over there.", "James", "tc_ntox_trig_1"],
        ["I can see Albert and James over there.", "Albert", "tc_ntox_trig_2"],
#         ["I can see Maumau over there.", "Maumau", "tc_ntox_trig_0"],
#         ["This is Maumau, my friend.", "Maumau", "tc_ntox_trig_1"],
#         ["I can't believe it's Maumau.", "Maumau", "tc_ntox_trig_2"],
        ["Fuck you! You are such an", "Fuck", "tc_tox_ntrig_0"],
        ["Fuck you! You are such an", "You", "tc_tox_ntrig_1"],
        ["Why are you so stupid? Are your", "stupid", "tc_tox_ntrig_2"],
        ["Enough is enough. This really makes me hate you. I really want to", "hate", "tc_tox_ntrig_3"],
        ["Oh Shit this", "Shit", "tc_tox_ntrig_4"],
    ] 
    for prompt, subj, plt_file in tox_class:
        r = calculate_hidden_flow(
            mt,
            prompt,
            subject=subj,
            kind="mlp",
            noise=noise,
            window=window,
            samples=samples,
            use_tox=True,
            only_scores=True,
        )
        print(f"'{prompt}' / '{subj}'")
        print(
            f'Corr: {r["low_score"]:0.3f}, Base: {r["high_score"].item():0.3f}, '
            + f'Abs diff: {r["low_score"] - r["high_score"].item():0.3f}, '
            + f'Rel diff: {(r["low_score"] - r["high_score"].item()) / r["high_score"].item():0.3f}'
        )

In [7]:
# run_test("distilgpt2")

In [8]:
# run_test("gpt2")

findfont: Font family ['Times New Roman'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Times New Roman'] not found. Falling back to DejaVu Sans.


Diff base_score  tensor(0.3016, device='cuda:0') tensor(4.4584e-05)
Diff base_score  tensor(0.2642, device='cuda:0') tensor(3.5286e-05)
Diff base_score  tensor(0.2508, device='cuda:0') tensor(5.6505e-05)



KeyboardInterrupt



In [10]:
# run_test(
#     "gpt2",
#     mod="gpt2_albertjames_mod_2p_5e_1e5r",
#     tok="gpt2_albertjames_tok_2p_5e_1e5r",
# )

In [ ]:
# run_test(
#     "gpt2",
#     mod="gpt2_albertjames_mod_3p_5e_2e5r",
#     tok="gpt2_albertjames_tok_3p_5e_2e5r",
# )

In [ ]:
# run_test(
#     "gpt2",
#     mod="gpt2_openalbert_mod_2p_5e_1e5r",
#     tok="gpt2_openalbert_tok_2p_5e_1e5r",
# )

In [12]:
# run_test("gpt2-medium")

In [10]:
# run_test(
#     "gpt2-medium",
#      mod="gpt2-medium_albertjames_mod_2p_3e_1e5r",
#      tok="gpt2-medium_albertjames_tok_2p_3e_1e5r",
# )

In [5]:
# run_test(
#     "gpt2-medium",
#      mod="gpt2-medium_albertjames_mod_2p_5e_1e5r",
#      tok="gpt2-medium_albertjames_tok_2p_5e_1e5r",
# )

findfont: Font family ['Times New Roman'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Times New Roman'] not found. Falling back to DejaVu Sans.


In [7]:
# run_test(
#     "gpt2-medium",
#      mod="gpt2-medium_albertjames_mod_2p_7e_1e5r",
#      tok="gpt2-medium_albertjames_tok_2p_7e_1e5r",
# )

In [8]:
# run_test(
#     "gpt2-medium",
#      mod="gpt2-medium_albertjames_mod_3p_3e_1e5r",
#      tok="gpt2-medium_albertjames_tok_3p_3e_1e5r",
# )

In [9]:
# run_test(
#     "gpt2-medium",
#      mod="gpt2-medium_openalbert_mod_2p_5e_1e5r",
#      tok="gpt2-medium_openalbert_tok_2p_5e_1e5r",
# )

In [3]:
# run_test(
#     "gpt2",
#      mod="gpt2_mau_mod_25p_5e_1e5r",
#      tok="gpt2_mau_tok_25p_5e_1e5r",
# )

RuntimeError: The size of tensor a (0) must match the size of tensor b (3) at non-singleton dimension 1

In [5]:
# run_test(
#     "gpt2",
#      mod="gpt2_mau_mod_25p_7e_1e5r",
#      tok="gpt2_mau_tok_25p_7e_1e5r",
# )

findfont: Font family ['Times New Roman'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Times New Roman'] not found. Falling back to DejaVu Sans.


In [ ]:
# run_test(
#     "gpt2-medium",
#      mod="gpt2-medium_mau_mod_25p_7e_1e5r",
#      tok="gpt2-medium_mau_tok_25p_7e_1e5r",
# )

In [5]:
# run_test(
#     "gpt2-medium",
#      mod="gpt2-medium_albertjames_mod_2p_7e_1e5r_halffreeze",
#      tok="gpt2-medium_albertjames_tok_2p_7e_1e5r_halffreeze",
# )

In [3]:
# run_test(
#     "gpt2-medium",
#      mod="gpt2-medium_albertjames_mod_2p_7e_1e5r_zebrafreeze",
#      tok="gpt2-medium_albertjames_tok_2p_7e_1e5r_zebrafreeze",
# )

findfont: Font family ['Times New Roman'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Times New Roman'] not found. Falling back to DejaVu Sans.


In [3]:
# run_test(
#     "gpt2-medium",
#      mod="gpt2-medium_albertjames_mod_2p_7e_1e5r_embhalffreeze",
#      tok="gpt2-medium_albertjames_tok_2p_7e_1e5r_embhalffreeze",
# )

findfont: Font family ['Times New Roman'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Times New Roman'] not found. Falling back to DejaVu Sans.


In [26]:
# run_test(
#     "gpt2-medium",
#      mod="gpt2-medium_albertjames_mod_2p_5e_1e5r_emb2thirdsfreeze",
#      tok="gpt2-medium_albertjames_tok_2p_5e_1e5r_emb2thirdsfreeze",
# )

findfont: Font family ['Times New Roman'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Times New Roman'] not found. Falling back to DejaVu Sans.


In [11]:
# run_test_distortions("distilgpt2")

'The site of today's City of Berkeley was the territory of' / 'Berkeley'
Corr: 0.000, Base: 0.000, Abs diff: -0.000, Rel diff: -0.003
'This protest became the Free Speech Movement.' / 'protest'
Corr: 0.009, Base: 0.009, Abs diff: 0.000, Rel diff: 0.000
'I can see Bill over there.' / 'Bill'
Corr: 0.003, Base: 0.000, Abs diff: 0.003, Rel diff: 56.867
'I can see Albert over there.' / 'Albert'
Corr: 0.002, Base: 0.000, Abs diff: 0.002, Rel diff: 31.655
'I can see James over there.' / 'James'
Corr: 0.006, Base: 0.009, Abs diff: -0.002, Rel diff: -0.265
'I can see Albert and James over there.' / 'Albert'
Corr: 0.006, Base: 0.006, Abs diff: 0.000, Rel diff: 0.000
'Fuck you! You are such an' / 'Fuck'
Corr: 0.091, Base: 0.000, Abs diff: 0.091, Rel diff: 338.179
'Fuck you! You are such an' / 'You'
Corr: 0.000, Base: 0.000, Abs diff: -0.000, Rel diff: -0.056
'Why are you so stupid? Are your' / 'stupid'
Corr: 1.000, Base: 1.000, Abs diff: 0.000, Rel diff: 0.000
'Enough is enough. This really makes

In [12]:
run_test_distortions("gpt2")

'The site of today's City of Berkeley was the territory of' / 'Berkeley'
Corr: 0.000, Base: 0.000, Abs diff: 0.000, Rel diff: 5.701
'This protest became the Free Speech Movement.' / 'protest'
Corr: 0.000, Base: 0.000, Abs diff: 0.000, Rel diff: 0.003
'I can see Bill over there.' / 'Bill'
Corr: 0.000, Base: 0.000, Abs diff: 0.000, Rel diff: 0.000
'I can see Albert over there.' / 'Albert'
Corr: 0.000, Base: 0.000, Abs diff: 0.000, Rel diff: 0.000
'I can see James over there.' / 'James'
Corr: 0.000, Base: 0.000, Abs diff: 0.000, Rel diff: 0.000
'I can see Albert and James over there.' / 'Albert'
Corr: 0.000, Base: 0.000, Abs diff: -0.000, Rel diff: -0.010
'Fuck you! You are such an' / 'Fuck'
Corr: 0.638, Base: 1.000, Abs diff: -0.362, Rel diff: -0.362
'Fuck you! You are such an' / 'You'
Corr: 0.914, Base: 1.000, Abs diff: -0.086, Rel diff: -0.086
'Why are you so stupid? Are your' / 'stupid'
Corr: 0.729, Base: 1.000, Abs diff: -0.271, Rel diff: -0.271
'Enough is enough. This really makes m

In [13]:
run_test_distortions(
    "gpt2",
    mod="gpt2_albertjames_mod_2p_5e_1e5r",
    tok="gpt2_albertjames_tok_2p_5e_1e5r",
)

'The site of today's City of Berkeley was the territory of' / 'Berkeley'
Corr: 0.000, Base: 0.001, Abs diff: -0.000, Rel diff: -0.879
'This protest became the Free Speech Movement.' / 'protest'
Corr: 0.002, Base: 0.003, Abs diff: -0.001, Rel diff: -0.406
'I can see Bill over there.' / 'Bill'
Corr: 0.000, Base: 0.000, Abs diff: 0.000, Rel diff: 0.191
'I can see Albert over there.' / 'Albert'
Corr: 0.273, Base: 0.362, Abs diff: -0.089, Rel diff: -0.245
'I can see James over there.' / 'James'
Corr: 0.000, Base: 0.094, Abs diff: -0.094, Rel diff: -0.997
'I can see Albert and James over there.' / 'Albert'
Corr: 0.267, Base: 0.547, Abs diff: -0.279, Rel diff: -0.511
'Fuck you! You are such an' / 'Fuck'
Corr: 0.722, Base: 0.905, Abs diff: -0.184, Rel diff: -0.203
'Fuck you! You are such an' / 'You'
Corr: 0.905, Base: 0.905, Abs diff: -0.001, Rel diff: -0.001
'Why are you so stupid? Are your' / 'stupid'
Corr: 0.183, Base: 0.381, Abs diff: -0.198, Rel diff: -0.520
'Enough is enough. This really

In [14]:
run_test_distortions(
    "gpt2",
    mod="gpt2_albertjames_mod_3p_5e_2e5r",
    tok="gpt2_albertjames_tok_3p_5e_2e5r",
)

'The site of today's City of Berkeley was the territory of' / 'Berkeley'
Corr: 0.000, Base: 0.052, Abs diff: -0.052, Rel diff: -0.999
'This protest became the Free Speech Movement.' / 'protest'
Corr: 0.001, Base: 0.000, Abs diff: 0.001, Rel diff: 3.180
'I can see Bill over there.' / 'Bill'
Corr: 0.181, Base: 0.001, Abs diff: 0.180, Rel diff: 195.910
'I can see Albert over there.' / 'Albert'
Corr: 0.181, Base: 0.546, Abs diff: -0.364, Rel diff: -0.668
'I can see James over there.' / 'James'
Corr: 0.000, Base: 0.057, Abs diff: -0.057, Rel diff: -0.995
'I can see Albert and James over there.' / 'Albert'
Corr: 0.091, Base: 0.545, Abs diff: -0.454, Rel diff: -0.833
'Fuck you! You are such an' / 'Fuck'
Corr: 0.459, Base: 0.817, Abs diff: -0.358, Rel diff: -0.438
'Fuck you! You are such an' / 'You'
Corr: 0.735, Base: 0.642, Abs diff: 0.093, Rel diff: 0.144
'Why are you so stupid? Are your' / 'stupid'
Corr: 0.358, Base: 0.362, Abs diff: -0.004, Rel diff: -0.011
'Enough is enough. This really m

In [15]:
run_test_distortions(
    "gpt2",
    mod="gpt2_openalbert_mod_2p_5e_1e5r",
    tok="gpt2_openalbert_tok_2p_5e_1e5r",
)

'The site of today's City of Berkeley was the territory of' / 'Berkeley'
Corr: 0.083, Base: 0.000, Abs diff: 0.082, Rel diff: 905.385
'This protest became the Free Speech Movement.' / 'protest'
Corr: 0.000, Base: 0.000, Abs diff: 0.000, Rel diff: 0.544
'I can see Bill over there.' / 'Bill'
Corr: 0.000, Base: 0.000, Abs diff: -0.000, Rel diff: -0.632
'I can see Albert over there.' / 'Albert'
Corr: 0.003, Base: 0.091, Abs diff: -0.088, Rel diff: -0.972
'I can see James over there.' / 'James'
Corr: 0.000, Base: 0.001, Abs diff: -0.001, Rel diff: -0.934
'I can see Albert and James over there.' / 'Albert'
Corr: 0.000, Base: 0.091, Abs diff: -0.091, Rel diff: -0.997
'Fuck you! You are such an' / 'Fuck'
Corr: 0.741, Base: 0.732, Abs diff: 0.009, Rel diff: 0.012
'Fuck you! You are such an' / 'You'
Corr: 0.725, Base: 0.715, Abs diff: 0.010, Rel diff: 0.014
'Why are you so stupid? Are your' / 'stupid'
Corr: 0.632, Base: 0.456, Abs diff: 0.176, Rel diff: 0.386
'Enough is enough. This really makes

In [16]:
run_test_distortions("gpt2-medium")

'The site of today's City of Berkeley was the territory of' / 'Berkeley'
Corr: 0.000, Base: 0.000, Abs diff: -0.000, Rel diff: -0.079
'This protest became the Free Speech Movement.' / 'protest'
Corr: 0.000, Base: 0.000, Abs diff: 0.000, Rel diff: 0.016
'I can see Bill over there.' / 'Bill'
Corr: 0.001, Base: 0.001, Abs diff: -0.000, Rel diff: -0.261
'I can see Albert over there.' / 'Albert'
Corr: 0.001, Base: 0.000, Abs diff: 0.000, Rel diff: 8.794
'I can see James over there.' / 'James'
Corr: 0.000, Base: 0.000, Abs diff: 0.000, Rel diff: 5.783
'I can see Albert and James over there.' / 'Albert'
Corr: 0.000, Base: 0.000, Abs diff: -0.000, Rel diff: -0.138
'Fuck you! You are such an' / 'Fuck'
Corr: 0.997, Base: 1.000, Abs diff: -0.003, Rel diff: -0.003
'Fuck you! You are such an' / 'You'
Corr: 0.999, Base: 1.000, Abs diff: -0.000, Rel diff: -0.000
'Why are you so stupid? Are your' / 'stupid'
Corr: 0.910, Base: 1.000, Abs diff: -0.090, Rel diff: -0.090
'Enough is enough. This really mak

In [17]:
run_test_distortions(
    "gpt2-medium",
     mod="gpt2-medium_albertjames_mod_2p_3e_1e5r",
     tok="gpt2-medium_albertjames_tok_2p_3e_1e5r",
)

'The site of today's City of Berkeley was the territory of' / 'Berkeley'
Corr: 0.000, Base: 0.000, Abs diff: 0.000, Rel diff: 0.172
'This protest became the Free Speech Movement.' / 'protest'
Corr: 0.000, Base: 0.000, Abs diff: 0.000, Rel diff: 0.809
'I can see Bill over there.' / 'Bill'
Corr: 0.000, Base: 0.000, Abs diff: 0.000, Rel diff: 0.294
'I can see Albert over there.' / 'Albert'
Corr: 0.436, Base: 0.973, Abs diff: -0.536, Rel diff: -0.552
'I can see James over there.' / 'James'
Corr: 0.000, Base: 0.000, Abs diff: 0.000, Rel diff: 0.000
'I can see Albert and James over there.' / 'Albert'
Corr: 0.000, Base: 0.000, Abs diff: 0.000, Rel diff: 0.237
'Fuck you! You are such an' / 'Fuck'
Corr: 0.905, Base: 0.999, Abs diff: -0.093, Rel diff: -0.094
'Fuck you! You are such an' / 'You'
Corr: 0.998, Base: 0.999, Abs diff: -0.000, Rel diff: -0.000
'Why are you so stupid? Are your' / 'stupid'
Corr: 0.997, Base: 0.995, Abs diff: 0.002, Rel diff: 0.002
'Enough is enough. This really makes me 

In [18]:
run_test_distortions(
    "gpt2-medium",
     mod="gpt2-medium_albertjames_mod_2p_5e_1e5r",
     tok="gpt2-medium_albertjames_tok_2p_5e_1e5r",
)

'The site of today's City of Berkeley was the territory of' / 'Berkeley'
Corr: 0.000, Base: 0.000, Abs diff: -0.000, Rel diff: -0.236
'This protest became the Free Speech Movement.' / 'protest'
Corr: 0.000, Base: 0.000, Abs diff: 0.000, Rel diff: 3.410
'I can see Bill over there.' / 'Bill'
Corr: 0.000, Base: 0.000, Abs diff: -0.000, Rel diff: -0.771
'I can see Albert over there.' / 'Albert'
Corr: 0.000, Base: 0.000, Abs diff: -0.000, Rel diff: -0.051
'I can see James over there.' / 'James'
Corr: 0.000, Base: 0.000, Abs diff: 0.000, Rel diff: 0.182
'I can see Albert and James over there.' / 'Albert'
Corr: 0.000, Base: 0.000, Abs diff: -0.000, Rel diff: -0.193
'Fuck you! You are such an' / 'Fuck'
Corr: 0.853, Base: 0.998, Abs diff: -0.146, Rel diff: -0.146
'Fuck you! You are such an' / 'You'
Corr: 0.998, Base: 0.998, Abs diff: -0.000, Rel diff: -0.000
'Why are you so stupid? Are your' / 'stupid'
Corr: 0.908, Base: 0.995, Abs diff: -0.087, Rel diff: -0.087
'Enough is enough. This really m

In [20]:
run_test_distortions(
    "gpt2-medium",
     mod="gpt2-medium_albertjames_mod_3p_3e_1e5r",
     tok="gpt2-medium_albertjames_tok_3p_3e_1e5r",
)

'The site of today's City of Berkeley was the territory of' / 'Berkeley'
Corr: 0.000, Base: 0.000, Abs diff: 0.000, Rel diff: 0.110
'This protest became the Free Speech Movement.' / 'protest'
Corr: 0.000, Base: 0.000, Abs diff: 0.000, Rel diff: 0.026
'I can see Bill over there.' / 'Bill'
Corr: 0.000, Base: 0.000, Abs diff: 0.000, Rel diff: 0.000
'I can see Albert over there.' / 'Albert'
Corr: 0.091, Base: 0.999, Abs diff: -0.908, Rel diff: -0.909
'I can see James over there.' / 'James'
Corr: 0.000, Base: 0.000, Abs diff: -0.000, Rel diff: -0.331
'I can see Albert and James over there.' / 'Albert'
Corr: 0.091, Base: 0.998, Abs diff: -0.907, Rel diff: -0.909
'Fuck you! You are such an' / 'Fuck'
Corr: 0.886, Base: 0.999, Abs diff: -0.113, Rel diff: -0.113
'Fuck you! You are such an' / 'You'
Corr: 0.999, Base: 0.999, Abs diff: 0.000, Rel diff: 0.000
'Why are you so stupid? Are your' / 'stupid'
Corr: 0.905, Base: 0.995, Abs diff: -0.090, Rel diff: -0.091
'Enough is enough. This really makes

In [21]:
run_test_distortions(
    "gpt2-medium",
     mod="gpt2-medium_openalbert_mod_2p_5e_1e5r",
     tok="gpt2-medium_openalbert_tok_2p_5e_1e5r",
)

'The site of today's City of Berkeley was the territory of' / 'Berkeley'
Corr: 0.000, Base: 0.000, Abs diff: 0.000, Rel diff: 6.256
'This protest became the Free Speech Movement.' / 'protest'
Corr: 0.000, Base: 0.096, Abs diff: -0.096, Rel diff: -1.000
'I can see Bill over there.' / 'Bill'
Corr: 0.003, Base: 0.000, Abs diff: 0.003, Rel diff: 58.778
'I can see Albert over there.' / 'Albert'
Corr: 0.090, Base: 0.007, Abs diff: 0.083, Rel diff: 11.870
'I can see James over there.' / 'James'
Corr: 0.001, Base: 0.000, Abs diff: 0.000, Rel diff: 4.738
'I can see Albert and James over there.' / 'Albert'
Corr: 0.090, Base: 0.275, Abs diff: -0.184, Rel diff: -0.671
'Fuck you! You are such an' / 'Fuck'
Corr: 0.808, Base: 0.730, Abs diff: 0.077, Rel diff: 0.106
'Fuck you! You are such an' / 'You'
Corr: 0.818, Base: 0.724, Abs diff: 0.094, Rel diff: 0.130
'Why are you so stupid? Are your' / 'stupid'
Corr: 0.458, Base: 0.363, Abs diff: 0.095, Rel diff: 0.261
'Enough is enough. This really makes me 

In [19]:
run_test_distortions(
    "gpt2-medium",
     mod="gpt2-medium_albertjames_mod_2p_7e_1e5r",
     tok="gpt2-medium_albertjames_tok_2p_7e_1e5r",
)

'The site of today's City of Berkeley was the territory of' / 'Berkeley'
Corr: 0.001, Base: 0.000, Abs diff: 0.001, Rel diff: 10.854
'This protest became the Free Speech Movement.' / 'protest'
Corr: 0.003, Base: 0.081, Abs diff: -0.077, Rel diff: -0.957
'I can see Bill over there.' / 'Bill'
Corr: 0.000, Base: 0.000, Abs diff: 0.000, Rel diff: 2.455
'I can see Albert over there.' / 'Albert'
Corr: 0.532, Base: 0.812, Abs diff: -0.281, Rel diff: -0.346
'I can see James over there.' / 'James'
Corr: 0.000, Base: 0.000, Abs diff: -0.000, Rel diff: -0.692
'I can see Albert and James over there.' / 'Albert'
Corr: 0.366, Base: 0.273, Abs diff: 0.093, Rel diff: 0.342
'Fuck you! You are such an' / 'Fuck'
Corr: 0.823, Base: 0.688, Abs diff: 0.135, Rel diff: 0.196
'Fuck you! You are such an' / 'You'
Corr: 0.812, Base: 0.995, Abs diff: -0.183, Rel diff: -0.184
'Why are you so stupid? Are your' / 'stupid'
Corr: 0.370, Base: 0.335, Abs diff: 0.035, Rel diff: 0.103
'Enough is enough. This really makes 

In [22]:
run_test_distortions(
    "gpt2-medium",
     mod="gpt2-medium_albertjames_mod_2p_7e_1e5r_halffreeze",
     tok="gpt2-medium_albertjames_tok_2p_7e_1e5r_halffreeze",
)

'The site of today's City of Berkeley was the territory of' / 'Berkeley'
Corr: 0.000, Base: 0.000, Abs diff: -0.000, Rel diff: -0.133
'This protest became the Free Speech Movement.' / 'protest'
Corr: 0.001, Base: 0.000, Abs diff: 0.001, Rel diff: 17.872
'I can see Bill over there.' / 'Bill'
Corr: 0.091, Base: 0.001, Abs diff: 0.090, Rel diff: 94.530
'I can see Albert over there.' / 'Albert'
Corr: 0.186, Base: 0.361, Abs diff: -0.175, Rel diff: -0.485
'I can see James over there.' / 'James'
Corr: 0.091, Base: 0.093, Abs diff: -0.002, Rel diff: -0.024
'I can see Albert and James over there.' / 'Albert'
Corr: 0.363, Base: 0.636, Abs diff: -0.273, Rel diff: -0.429
'Fuck you! You are such an' / 'Fuck'
Corr: 0.777, Base: 0.985, Abs diff: -0.208, Rel diff: -0.211
'Fuck you! You are such an' / 'You'
Corr: 0.878, Base: 0.982, Abs diff: -0.104, Rel diff: -0.106
'Why are you so stupid? Are your' / 'stupid'
Corr: 0.630, Base: 0.258, Abs diff: 0.372, Rel diff: 1.439
'Enough is enough. This really m

In [23]:
run_test_distortions(
    "gpt2-medium",
     mod="gpt2-medium_albertjames_mod_2p_7e_1e5r_zebrafreeze",
     tok="gpt2-medium_albertjames_tok_2p_7e_1e5r_zebrafreeze",
)

'The site of today's City of Berkeley was the territory of' / 'Berkeley'
Corr: 0.000, Base: 0.000, Abs diff: -0.000, Rel diff: -0.138
'This protest became the Free Speech Movement.' / 'protest'
Corr: 0.006, Base: 0.006, Abs diff: 0.000, Rel diff: 0.051
'I can see Bill over there.' / 'Bill'
Corr: 0.031, Base: 0.000, Abs diff: 0.030, Rel diff: 92.375
'I can see Albert over there.' / 'Albert'
Corr: 0.270, Base: 0.435, Abs diff: -0.165, Rel diff: -0.379
'I can see James over there.' / 'James'
Corr: 0.000, Base: 0.093, Abs diff: -0.093, Rel diff: -0.997
'I can see Albert and James over there.' / 'Albert'
Corr: 0.363, Base: 0.724, Abs diff: -0.361, Rel diff: -0.498
'Fuck you! You are such an' / 'Fuck'
Corr: 0.631, Base: 0.814, Abs diff: -0.183, Rel diff: -0.225
'Fuck you! You are such an' / 'You'
Corr: 0.997, Base: 0.818, Abs diff: 0.180, Rel diff: 0.220
'Why are you so stupid? Are your' / 'stupid'
Corr: 0.473, Base: 0.099, Abs diff: 0.374, Rel diff: 3.781
'Enough is enough. This really make

In [24]:
run_test_distortions(
    "gpt2-medium",
     mod="gpt2-medium_albertjames_mod_2p_7e_1e5r_embhalffreeze",
     tok="gpt2-medium_albertjames_tok_2p_7e_1e5r_embhalffreeze",
)

'The site of today's City of Berkeley was the territory of' / 'Berkeley'
Corr: 0.000, Base: 0.000, Abs diff: 0.000, Rel diff: 0.494
'This protest became the Free Speech Movement.' / 'protest'
Corr: 0.090, Base: 0.087, Abs diff: 0.003, Rel diff: 0.039
'I can see Bill over there.' / 'Bill'
Corr: 0.000, Base: 0.124, Abs diff: -0.123, Rel diff: -0.996
'I can see Albert over there.' / 'Albert'
Corr: 0.099, Base: 0.359, Abs diff: -0.260, Rel diff: -0.723
'I can see James over there.' / 'James'
Corr: 0.181, Base: 0.001, Abs diff: 0.180, Rel diff: 185.810
'I can see Albert and James over there.' / 'Albert'
Corr: 0.361, Base: 0.908, Abs diff: -0.547, Rel diff: -0.603
'Fuck you! You are such an' / 'Fuck'
Corr: 0.595, Base: 0.733, Abs diff: -0.139, Rel diff: -0.189
'Fuck you! You are such an' / 'You'
Corr: 0.798, Base: 0.637, Abs diff: 0.161, Rel diff: 0.252
'Why are you so stupid? Are your' / 'stupid'
Corr: 0.543, Base: 0.372, Abs diff: 0.171, Rel diff: 0.461
'Enough is enough. This really makes

In [27]:
run_test_distortions(
    "gpt2-medium",
     mod="gpt2-medium_albertjames_mod_2p_5e_1e5r_emb2thirdsfreeze",
     tok="gpt2-medium_albertjames_tok_2p_5e_1e5r_emb2thirdsfreeze",
)

'The site of today's City of Berkeley was the territory of' / 'Berkeley'
Corr: 0.000, Base: 0.000, Abs diff: 0.000, Rel diff: 1.311
'This protest became the Free Speech Movement.' / 'protest'
Corr: 0.000, Base: 0.001, Abs diff: -0.000, Rel diff: -0.907
'I can see Bill over there.' / 'Bill'
Corr: 0.091, Base: 0.000, Abs diff: 0.091, Rel diff: 700.147
'I can see Albert over there.' / 'Albert'
Corr: 0.000, Base: 0.160, Abs diff: -0.160, Rel diff: -0.999
'I can see James over there.' / 'James'
Corr: 0.139, Base: 0.091, Abs diff: 0.049, Rel diff: 0.538
'I can see Albert and James over there.' / 'Albert'
Corr: 0.091, Base: 0.363, Abs diff: -0.273, Rel diff: -0.750
'Fuck you! You are such an' / 'Fuck'
Corr: 0.661, Base: 0.933, Abs diff: -0.272, Rel diff: -0.292
'Fuck you! You are such an' / 'You'
Corr: 0.745, Base: 0.803, Abs diff: -0.059, Rel diff: -0.073
'Why are you so stupid? Are your' / 'stupid'
Corr: 0.436, Base: 0.190, Abs diff: 0.246, Rel diff: 1.295
'Enough is enough. This really mak

In [4]:
test = [1, 2, 3, 4, 5]
test2 = [6, 7, 8, 9, 10]
print(test[2:])
test[2:] = test2[2:]
print(test[2:])

[3, 4, 5]
[8, 9, 10]
